### 학습 목표

- 새로운 데이터가 주어지면, 우리는 어떻게 데이터와 친해져야 할까요? (EDA process)


- 데이터가 대용량일 때, 특정 기술을 포함한 다양한 방법으로 어떻게 해결해야 할까요?


- 여러 개의 데이터를 합쳐서 평가할 때는 어떤 기준을 사용해야 할까요?


- 공부했던 예시들을 가지고, 여러분 만의 EDA Notebook을 만들어보세요!

## ASHRAE ENERGY PREDICTION III

- 건물 에너지 사용량 예측

- 분석 문제 정의에 따라 시계열 분석 방법으로도 분석이 가능하다.
(우리는 시계열 분석으로 풀지는 않습니다)

- 주어진 건물별 에너지 사용량 정보와, 날씨 정보를 통해서 건물별 에너지 사용량 예측 문제를 풀어봅시다.

[Data] https://www.kaggle.com/c/ashrae-energy-prediction/data

### 데이터 및 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

data_path = "../input/ashrae-energy-prediction/"

train_path = data_path + "train.csv"
#test_path = data_path + "test.csv"

building_path = data_path + "building_metadata.csv"

weather_train_path = data_path + "weather_train.csv"
#weather_test_path = data_path + "weather_test.csv"

### ***질문 환영!!!***

In [ ]:
train_data = pd.read_csv(train_path)
#test_data = pd.read_csv(test_path)

building_data = pd.read_csv(building_path)

weather_train_data = pd.read_csv(weather_train_path)
#weather_test_data = pd.read_csv(weather_test_path)
print(train_data.shape, building_data.shape, weather_train_data.shape)

### (OPTIONAL) Reducing the memory usage of the data

- ASHRAE 데이터는 데이터셋이 매우 큽니다. 메모리를 줄여서 처리하기 편하게 만듭니다.


- 이전 실습교안에서 만들었던 함수를 불러와서 사용합시다.

In [ ]:
## Reducing DataFrame memory size by ~65%
#####
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size\n\n")
    return props, NAlist

In [ ]:
## apply
train_data, _ = reduce_mem_usage(train_data)
weather_train_data, _ = reduce_mem_usage(weather_train_data)
building_data, _ = reduce_mem_usage(building_data)

### Do your Own EDA

- 각자의 방법대로 EDA를 해봅시다.


- EDA를 할 때 중요한 사항은, "내가 이 데이터에서 보고 싶은 포인트가 무엇인가?" 입니다.


- EDA에서 정답은 없습니다. 다양한 케이스를 살펴보면서 감을 익힐뿐입니다.


- 전혀 감을 잡지 못하겠다고 하면, kaggle에 있는 좋은 EDA notebook을 필사해보는 것도 좋습니다.

> 추천 1. https://www.kaggle.com/code/caesarlupum/ashrae-start-here-a-gentle-introduction

> 추천 2. https://www.kaggle.com/code/jaseziv83/a-deep-dive-eda-into-all-variables

> 추천 3. https://www.kaggle.com/code/hmendonca/starter-eda-and-feature-selection-ashrae3

In [ ]:
for col in train_data.columns: # 각 column마다
    print(f"{col}:\t{train_data[col].isnull().mean() * 100:.2f}%") # column별 결측치 비율

In [ ]:
for col in building_data.columns: # 각 column마다
    print(f"{col}:\t{building_data[col].isnull().mean() * 100:.2f}%") # column별 결측치 비율

In [ ]:
for col in weather_train_data.columns: # 각 column마다
    print(f"{col}:\t{weather_train_data[col].isnull().mean() * 100:.2f}%") # column별 결측치 비율

In [ ]:
# 데이터를 모두 합쳐보세요!
full_data = pd.merge(train_data, building_data, on="building_id")
full_data = pd.merge(full_data, weather_train_data, on=["site_id", "timestamp"], how="left")
full_data

In [ ]:
full_data.info()

In [ ]:
full_data[full_data.isnull().any(axis=1)]

In [ ]:
for col in full_data.columns: # 각 column마다
    print(f"{col}:\t{full_data[col].isnull().mean() * 100:.2f}%") # column별 결측치 비율

### Conclusion

- 여러분들의 내린 이 데이터에서 중요한 지표는 무엇인가요?


- 가장 중요하다고 생각하는 그래프 3개를 제출해주세요. (핵심 포인트가 들어간 그래프)
(plt.savefig 함수를 사용합니다)


- Energy prediction을 하기 위해서 어떤 feature를 사용하는 것이 좋은 방법인지 생각을 정리해보세요.